#**Цель: выполнить полноценный исследовательский анализ данных (EDA), применяя методы выявления и обработки пропусков, выбросов, масштабирования числовых признаков и кодирования категориальных переменных.**
1. Выберите реальный табличный датасет со структурой не менее 8
признаков и целевым признаком, подходящим для задач регрессии или
классификации.
Возможные варианты:
датасеты с платформ Kaggle, UCI, OpenML;
встроенные датасеты из библиотек sklearn , seaborn ;
или запросите рекомендованный набор у преподавателя.
2. Убедитесь, что в датасете присутствуют:
числовые признаки,
категориальные признаки,
целевой столбец (предсказуемый признак),
необязательно: пропущенные значения или выбросы
Выполните последовательные шаги исследовательского анализа:
1. Обзор структуры данных
Загрузите датасет.
Выведите общую информацию ( .info() , .describe() ).
Опишите: сколько признаков, каких типов, какова структура целевого признака
2. Обнаружение и обработка пропусков
Определите, есть ли пропущенные значения.
Обоснуйте выбранный способ их устранения (удаление, заполнение средним/
модой и т.д.).
Примените выбранный способ
3. Обнаружение и удаление выбросов
Выберите 3–5 числовых признаков.
Используя метод IQR, удалите выбросы
Сравните объём данных до и после очистки
4. Масштабирование числовых признаков
Выполните стандартизацию (z-преобразование) с помощью StandardScaler .
Объясните, зачем выполняется масштабирование
5. Кодирование категориальных признаков
Выполните:
Label Encoding для порядковых признаков (при наличии);
One-Hot Encoding для номинальных признаков.
Проверьте, исключена ли дамми-ловушка
6. Финальный набор данных
Убедитесь, что датасет не содержит пропусков, выбросов, категориальных
данных в строковом виде.
Признаки приведены к числовому виду, масштабированы.
Представьте итоговый DataFrame , готовый к использованию в моделях. избегать слишком простых или широко разобранных наборов ( iris ,
titanic , diamonds

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# 1. Загрузка данных
# Скачиваем с UCI: https://archive.ics.uci.edu/ml/datasets/Adult
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num',
    'marital_status', 'occupation', 'relationship', 'race', 'sex',
    'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income'
]

url_train = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
url_test = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test'

df_train = pd.read_csv(url_train, header=None, names=column_names, na_values=' ?', skipinitialspace=True)
df_test = pd.read_csv(url_test, header=0, names=column_names, na_values=' ?', skipinitialspace=True, comment='|')

df = pd.concat([df_train, df_test], ignore_index=True)

# 1. Обзор структуры данных
print("Общая информация о датасете:")
print(df.info())

print("\nОписательная статистика по числовым признакам:")
print(df.describe())

print("\nРаспределение целевого признака 'income':")
print(df['income'].value_counts())

# 2. Обнаружение и обработка пропусков
print("\nПропуски в каждом столбце:")
print(df.isnull().sum())

# Есть пропуски в workclass, occupation, native_country
# Решение: для категориальных признаков заполним модой

for col in ['workclass', 'occupation', 'native_country']:
    mode = df[col].mode()[0]
    df[col].fillna(mode, inplace=True)

print("\nПропуски после заполнения:")
print(df.isnull().sum())

# 3. Обнаружение и удаление выбросов
# Выберем числовые признаки: age, fnlwgt, education_num, capital_gain, capital_loss, hours_per_week

def remove_outliers_iqr(df, cols):
    df_clean = df.copy()
    initial_len = len(df_clean)
    for col in cols:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df_clean = df_clean[(df_clean[col] >= lower) & (df_clean[col] <= upper)]
    print(f"Длина до очистки: {initial_len}, после очистки: {len(df_clean)}")
    return df_clean

num_cols = ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']
df_clean = remove_outliers_iqr(df, num_cols)

# 4. Масштабирование числовых признаков
scaler = StandardScaler()
df_clean[num_cols] = scaler.fit_transform(df_clean[num_cols])

# 5. Кодирование категориальных признаков
# Определим порядковый признак — 'education' (образование) можно считать порядковым

# Создадим словарь с порядком для 'education'
education_order = [
    'Preschool', '1st-4th', '5th-6th', '7th-8th', '9th',
    '10th', '11th', '12th', 'HS-grad', 'Some-college',
    'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Masters',
    'Doctorate', 'Prof-school'
]

edu_map = {k: v for v, k in enumerate(education_order)}
df_clean['education'] = df_clean['education'].map(edu_map)

# Остальные категориальные — номинальные
cat_nominal = ['workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']

df_final = pd.get_dummies(df_clean, columns=cat_nominal, drop_first=True)

# Проверяем дамми-ловушку — drop_first=True исключает избыточные признаки

# 6. Финальный набор данных
print("\nПроверка пропусков и типов в итоговом датасете:")
print(df_final.isnull().sum().sum())  # должно быть 0
print(df_final.dtypes.value_counts())

print("\nРазмер итогового DataFrame:", df_final.shape)

print("\nПример итоговых данных:")
print(df_final.head())

Общая информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48841 entries, 0 to 48840
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48841 non-null  int64 
 1   workclass       48841 non-null  object
 2   fnlwgt          48841 non-null  int64 
 3   education       48841 non-null  object
 4   education_num   48841 non-null  int64 
 5   marital_status  48841 non-null  object
 6   occupation      48841 non-null  object
 7   relationship    48841 non-null  object
 8   race            48841 non-null  object
 9   sex             48841 non-null  object
 10  capital_gain    48841 non-null  int64 
 11  capital_loss    48841 non-null  int64 
 12  hours_per_week  48841 non-null  int64 
 13  native_country  48841 non-null  object
 14  income          48841 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB
None

Описательная статистика по числовым признакам:
           

<ipython-input-13-73fb3d6e94eb>:41: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mode, inplace=True)
